# Transformer 难点理解与实现

视频：[19、Transformer模型Encoder原理精讲及其PyTorch逐行实现](https://www.bilibili.com/video/BV1cP4y1V7GF/)

Transformer 特点：

+ 无先验假设（例如：局部关联性、有序建模性）
    + 它认为一个位置的输入与任何位置的输入都是有可能相关的，因此没有任何先验假设。
    + 好处：相比 CNN 或 RNN，可以更快速地学习到无论长时的建模性还是短时的关联性。
+ 核心计算在于 self-attention，**平方**复杂度
    + 当序列很长时，计算的瓶颈就出在这里了。
    + 之后的很多改进都是改进 self-attention 的复杂度，这个过程其实就是向里面注入人为的先验假设，比如引入局部关联性
+ **数据量的要求与先验假设的程度成反比**
    + 先验假设越多，其实就是注入了越多的人为的经验，比如 CNN 的局部关联性，这对数据量的要求就会更低
    + 如果数据量很少的话，Transformer 并不会学到很好的东西。

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from loguru import logger
from typing import Optional

## Input Seq and Target Seq

这里先准备一下 input sequence 和 target sequence，由于不去真正的训练一个 Transformer，因此这里只是简单随机生成了 batch-size 为 2 的 demo 数据。

In [2]:
BATCH_SIZE = 2

# 单词表的大小
MAX_NUM_SRC_WORDS = 8  # source seq 的词表大小
MAX_NUM_TGT_WORDS = 8  # target seq 的词表大小
MODEL_DIM  = 8  # token 嵌入的 dim 大小，原论文是 512

# 序列的最大长度，未达到最大长度的 setence 会被填充 0
MAX_SRC_SEQ_LEN = 5
MAX_TGT_SEQ_LEN = 5
MAX_POSITION_LEN = 5  # 设置训练中一个样本经过 embedding 后序列的最大长度是 5

# seq len
src_len = torch.tensor([2, 4]).to(torch.int32)  # BACTH_SZIE 个元素，每个元素代表一个 seq 的长度
tgt_len = torch.tensor([4, 3]).to(torch.int32)

# 单词索引构成 source sentence 和 target sentence，并做了 padding，默认 pad 为 0
src_seq = [torch.randint(1, MAX_NUM_SRC_WORDS, size=(L, )) for L in src_len]  # 单词索引构成的句子
logger.info(f'src seq 填充前: {src_seq}')
src_seq = [F.pad(s, (0, max(src_len) - len(s))) for s in src_seq]  # 对 seq 进行 padding
logger.info(f'src seq 填充后: {src_seq}')
src_seq = torch.stack(src_seq)
logger.info(f'将 src_seq 转为 2d tensor:\n{src_seq}')

tgt_seq = [torch.randint(1, MAX_NUM_TGT_WORDS, size=(L, )) for L in tgt_len]
logger.info(f'tgt seq 填充前：{tgt_seq}')
tgt_seq = [F.pad(s, (0, max(tgt_len) - len(s))) for s in tgt_seq]
logger.info(f'tgt seq 填充后：{tgt_seq}')
tgt_seq = torch.stack(tgt_seq)
logger.info(f'将 tgt_seq 转为 2d tensor:\n{tgt_seq}')

2023-01-24 01:28:27.132 | INFO     | __main__:<module>:19 - src seq 填充前: [tensor([7, 2]), tensor([2, 2, 6, 3])]
2023-01-24 01:28:27.135 | INFO     | __main__:<module>:21 - src seq 填充后: [tensor([7, 2, 0, 0]), tensor([2, 2, 6, 3])]
2023-01-24 01:28:27.137 | INFO     | __main__:<module>:23 - 将 src_seq 转为 2d tensor:
tensor([[7, 2, 0, 0],
        [2, 2, 6, 3]])
2023-01-24 01:28:27.140 | INFO     | __main__:<module>:26 - tgt seq 填充前：[tensor([5, 2, 3, 3]), tensor([6, 2, 1])]
2023-01-24 01:28:27.142 | INFO     | __main__:<module>:28 - tgt seq 填充后：[tensor([5, 2, 3, 3]), tensor([6, 2, 1, 0])]
2023-01-24 01:28:27.144 | INFO     | __main__:<module>:30 - 将 tgt_seq 转为 2d tensor:
tensor([[5, 2, 3, 3],
        [6, 2, 1, 0]])


## Word Embedding

关于 Word embedding，以序列建模为例

考虑 source sentence 和 target sentence 构建序列，序列的字符以其在词表中的索引的形式表示

In [3]:
src_embedding_table = nn.Embedding(MAX_NUM_SRC_WORDS + 1, MODEL_DIM)  # +1 是因为多了一个 pad 的 0
tgt_embedding_table = nn.Embedding(MAX_NUM_TGT_WORDS + 1, MODEL_DIM)

logger.info(f'src_embedding:\n{src_embedding_table.weight.data}')  # 每一行是一个 embedding vector

2023-01-24 01:28:27.517 | INFO     | __main__:<module>:4 - src_embedding:
tensor([[-0.7424, -1.6105,  0.7115, -0.1593,  0.1111, -0.1441,  0.0062, -0.0502],
        [ 2.1749, -0.6337,  1.5355, -0.5491,  0.9168,  0.5688, -0.9439, -1.9222],
        [ 1.1948,  1.0590,  0.8754,  0.3469, -0.7505,  1.3855, -0.2001, -0.4663],
        [-0.1121, -0.1646,  0.2155,  1.3592,  1.4394, -0.4944,  0.3183, -1.5093],
        [ 0.2138, -0.7868, -1.7726,  1.1716, -0.1413, -0.5410,  1.6557, -2.2504],
        [ 0.3471,  0.5023, -0.9068,  1.5077,  0.2560,  0.4194, -0.1603, -0.3757],
        [ 0.5018, -0.4057,  0.5766, -0.6847,  1.1305,  0.5239,  0.2538, -1.0548],
        [-1.0822, -0.0597, -0.8225,  0.2625, -1.3272,  2.4932,  0.0279,  0.5452],
        [ 0.7321, -1.6830,  0.1498, -2.0971,  0.7634,  0.1356, -0.8378, -2.5869]])


In [4]:
src_embed = src_embedding_table(src_seq)  # [B, len, embed_dim]
tgt_embed = tgt_embedding_table(tgt_seq)
logger.info(f'src_seq 的 embedding:\n{src_embed}')

2023-01-24 01:28:29.023 | INFO     | __main__:<module>:3 - src_seq 的 embedding:
tensor([[[-1.0822, -0.0597, -0.8225,  0.2625, -1.3272,  2.4932,  0.0279,
           0.5452],
         [ 1.1948,  1.0590,  0.8754,  0.3469, -0.7505,  1.3855, -0.2001,
          -0.4663],
         [-0.7424, -1.6105,  0.7115, -0.1593,  0.1111, -0.1441,  0.0062,
          -0.0502],
         [-0.7424, -1.6105,  0.7115, -0.1593,  0.1111, -0.1441,  0.0062,
          -0.0502]],

        [[ 1.1948,  1.0590,  0.8754,  0.3469, -0.7505,  1.3855, -0.2001,
          -0.4663],
         [ 1.1948,  1.0590,  0.8754,  0.3469, -0.7505,  1.3855, -0.2001,
          -0.4663],
         [ 0.5018, -0.4057,  0.5766, -0.6847,  1.1305,  0.5239,  0.2538,
          -1.0548],
         [-0.1121, -0.1646,  0.2155,  1.3592,  1.4394, -0.4944,  0.3183,
          -1.5093]]], grad_fn=<EmbeddingBackward0>)


## Position Embedding

根据原论文，position embedding 的公式如下：

+ $PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}})$
+ $PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})$

其中 $pos$ 是 position，$i$ 是 dimension。

这个 Embedding 也是一个 2d matrix，是一个 seq 的 PE，行数是一个 seq 的最大长度，列数就是 $d_{model}$。所以在公式的计算过程中，$pos$ 的值就是这是第几行，$i$ 的值就是这是第几列。$2i$ 就是偶数列，$2i+1$ 就是奇数列。

由于 Position Embedding 是一个常量，因此不需要参与训练，所以设置 `requires_grad=False`。

> 原文选择这种 PE 方式，是因为它具有较好的对称性和唯一性，每个位置的 embedding 是确定的。

In [5]:
# 先把公式中括号里面的东西表示出来
pos_mat = torch.arange(MAX_POSITION_LEN).reshape([-1, 1])  # 是一个每行都一样的 2d matrix，每一行的值是这行在 matrix 中的行数
logger.info(f'pos matrix:\n{pos_mat}')

i_mat = torch.arange(0, 8, step=2).reshape([1, -1]) / MODEL_DIM  # 公式中的 $2i / d_model$ 部分
i_mat = torch.pow(10000, i_mat)  # 公式中的 $10000^(...)$ 部分
logger.info(f'i_mat:\n{i_mat}')

2023-01-24 01:28:31.497 | INFO     | __main__:<module>:3 - pos matrix:
tensor([[0],
        [1],
        [2],
        [3],
        [4]])
2023-01-24 01:28:31.501 | INFO     | __main__:<module>:7 - i_mat:
tensor([[   1.,   10.,  100., 1000.]])


In [6]:
# 构建出 Position Embedding 实例
pe_embedding_table = torch.zeros(MAX_POSITION_LEN, MODEL_DIM)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat / i_mat)  # 为 PE 的偶数列赋值
pe_embedding_table[:, 1::2] = torch.cos(pos_mat / i_mat)  # 为 PE 的奇数列赋值

# 把这个 table 转换成 nn.Embedding 的实例
pe_embedding = nn.Embedding(MAX_POSITION_LEN, MODEL_DIM)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)

logger.info(f'Position Embedding Table:\n{pe_embedding_table}')
logger.info(f'Position Embedding:\n{pe_embedding.weight.data}')

2023-01-24 01:28:32.267 | INFO     | __main__:<module>:10 - Position Embedding Table:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01]])
2023-01-24 01:28:32.270 | INFO     | __main__:<module>:11 - Position Embedding:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,

In [7]:
# 计算我们的数据的位置编码
batch_sz = len(src_seq)
src_pos = torch.arange(max(src_len), dtype=torch.int32).unsqueeze(0).repeat(batch_sz, 1)  # 为每一个 seq 生成一个 arange，这个 arange 代表每个 token 的位置编号，shape: [B, len]
tgt_pos = torch.arange(max(src_len), dtype=torch.int32).unsqueeze(0).repeat(batch_sz, 1)
logger.info(f'src_pos:\n{src_pos}')

src_pe_embed = pe_embedding(src_pos)  # seq 的位置 embedding，shape: [B, len, d_model]
tgt_pe_embed = pe_embedding(tgt_pos)
logger.info(f'src_pe_embed:\n{src_pe_embed}')

2023-01-24 01:28:32.842 | INFO     | __main__:<module>:5 - src_pos:
tensor([[0, 1, 2, 3],
        [0, 1, 2, 3]], dtype=torch.int32)
2023-01-24 01:28:32.847 | INFO     | __main__:<module>:9 - src_pe_embed:
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867

## Encoder Self-Attention Mask

$Attention(Q, K, V) = softmax (\frac{QK^T}{\sqrt{d_k}})V$

### 为什么需要做 scale

做 scaled 的原因，是让经过 softmax 后概率分布的方差不要那么大，再往深处挖的话，就是希望不要让 Jacobian 矩阵的导数变成 0 了。

In [8]:
# 演示一下 sofmax 的效果

alpha1 = 0.1
alpha2 = 10
score = torch.randn(5)
prob1 = F.softmax(score * alpha1, dim=-1)
prob2 = F.softmax(score * alpha2, dim=-1)

print(f'score: {score}')
print(f'prob1: {prob1}')
print(f'prob2: {prob2}')  # 可以看到 prob2 中元素之间的方差就特别大，分布很不均匀

score: tensor([-0.0545,  0.7984, -0.1394,  1.0497, -0.5834])
prob1: tensor([0.1943, 0.2116, 0.1927, 0.2170, 0.1843])
prob2: tensor([1.4812e-05, 7.4918e-02, 6.3379e-06, 9.2506e-01, 7.4752e-08])


In [9]:
# 从 Jacobian 的角度来看一下
# Jacobian 就体现了中间梯度传播时的梯度

def softmax_func(score: torch.Tensor):
    return F.softmax(score, dim=-1)

jaco_mat1 = torch.autograd.functional.jacobian(softmax_func, score * alpha1)
jaco_mat2 = torch.autograd.functional.jacobian(softmax_func, score * alpha2)

print(f'Jacobian 1:\n{jaco_mat1}')  # 可以看到里面的梯度值很稳定
print(f'Jacobian 2:\n{jaco_mat2}')  # 可以看到这个矩阵中很多梯度值已经接近了 0，导致难以更新

Jacobian 1:
tensor([[ 0.1566, -0.0411, -0.0374, -0.0422, -0.0358],
        [-0.0411,  0.1668, -0.0408, -0.0459, -0.0390],
        [-0.0374, -0.0408,  0.1556, -0.0418, -0.0355],
        [-0.0422, -0.0459, -0.0418,  0.1699, -0.0400],
        [-0.0358, -0.0390, -0.0355, -0.0400,  0.1503]])
Jacobian 2:
tensor([[ 1.4812e-05, -1.1097e-06, -9.3876e-11, -1.3702e-05, -1.1072e-12],
        [-1.1097e-06,  6.9305e-02, -4.7482e-07, -6.9304e-02, -5.6002e-09],
        [-9.3876e-11, -4.7482e-07,  6.3378e-06, -5.8629e-06, -4.7377e-13],
        [-1.3702e-05, -6.9304e-02, -5.8629e-06,  6.9323e-02, -6.9150e-08],
        [-1.1072e-12, -5.6002e-09, -4.7377e-13, -6.9150e-08,  7.4752e-08]])


### 构建 mask

在 $Attention(Q, K, V) = softmax (\frac{QK^T}{\sqrt{d_k}})V$ 运算中，Q: [B, T, embed]，K: [B, T, embed]，因此 score $QK^T$ 的 shape 为 [B, T, T]。我们一般是把 mask 是在 softmax 里面，然后希望被 mask 的值在 softmax 里面被变成 -inf，这样经过 softmax 运算后就变成 0 了，这刚好符合我们的意图。

所以我们要构建一个 mask matrix，并让它与 score 相乘，mask matrix 的元素值要么为 1，要么为 -inf，并且 mask 的 shape 与 score 的维度是一样的。

首先要知道一个 seq 中哪些元素是有效的，哪些元素是 pad 上去的，因此先得到 `valid_encoder_pos` 来指示有效位置：

In [10]:
# mask 的 shape：[batch_size, max_src_len, max_src_len]，值为 1 或 -inf
valid_encoder_pos = torch.not_equal(src_seq, 0).to(torch.int32)  # 代表 src_seq 有效位置，每个元素指示了一个 seq 的有效位置，值为 1 代表有效，0 代表无效
logger.info(f'seq 的有效位置:\n{valid_encoder_pos}')  # 每行是一个 seq，元素为 0 代表这是存在一个 token，为 0 代表这是 pad 上去的

2023-01-24 01:28:39.030 | INFO     | __main__:<module>:3 - seq 的有效位置:
tensor([[1, 1, 0, 0],
        [1, 1, 1, 1]], dtype=torch.int32)


借助于 `valid_encoder_pos`，我们可以计算出一个矩阵，用来指示某两个 token 之间是否可以计算相似度，若两个 token 中存在 pad 上去的，那将无法计算相似度。

下面来计算这个关联矩阵 `valid_encoder_pos_matrix`，shape: [B, T, T]，在一个 sample 中，[i, j] 处的元素为 1 则代表 token i 与 token j 之间可以计算相似度。

In [11]:
valid_encoder_pos.unsqueeze_(2)  # 扩展一下维度，shape: [B, T, 1]
valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1, 2))  # 做矩阵相乘，shape: [B, T, T]
logger.info(f'句子内的关联矩阵:\n{valid_encoder_pos_matrix}')  # 每个 seq 对应里面一个 [T, T] 的矩阵，这个矩阵第 i 行第 j 列为 1 则代表这个 seq 的 token i 与 token j 可以计算相似度，若为 0 则无法计算相似度，因为其中某个 token 是 pad 上去的 

2023-01-24 01:28:40.945 | INFO     | __main__:<module>:3 - 句子内的关联矩阵:
tensor([[[1, 1, 0, 0],
         [1, 1, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]], dtype=torch.int32)


In [12]:
# 生成 mask 矩阵，False 代表不 mask，True 代表需要 mask
mask_encoder_self_attn = valid_encoder_pos_matrix == 0  # [B, T, T]
logger.info(f'Encoder Self-Attention 处的掩码矩阵:\n{mask_encoder_self_attn}')

2023-01-24 01:28:42.423 | INFO     | __main__:<module>:3 - Encoder Self-Attention 处的掩码矩阵:
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])


In [13]:
# mask 的使用示例：
score = torch.randn(BATCH_SIZE, max(src_len), max(src_len))
masked_score = score.masked_fill(mask_encoder_self_attn, -1e9)
prob = F.softmax(masked_score, -1)
print('src_len:\n', src_len)
print('score:\n', score)
print('masked_score:\n', masked_score)
print('prob:\n', prob)

src_len:
 tensor([2, 4], dtype=torch.int32)
score:
 tensor([[[-2.5580, -0.1970,  2.0863, -0.9242],
         [ 1.0685, -0.5446, -0.5381,  1.7170],
         [ 0.4200,  0.1955,  0.9417, -0.4696],
         [-0.2621, -0.6755,  1.4472, -1.5948]],

        [[ 0.0954,  1.8803,  1.3745,  0.9287],
         [ 1.3380, -0.5561,  1.8973,  1.0935],
         [ 1.4950,  1.3333, -1.8175, -0.0631],
         [ 1.0610, -0.0628,  0.8603, -0.2742]]])
masked_score:
 tensor([[[-2.5580e+00, -1.9702e-01, -1.0000e+09, -1.0000e+09],
         [ 1.0685e+00, -5.4455e-01, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[ 9.5446e-02,  1.8803e+00,  1.3745e+00,  9.2868e-01],
         [ 1.3380e+00, -5.5608e-01,  1.8973e+00,  1.0935e+00],
         [ 1.4950e+00,  1.3333e+00, -1.8175e+00, -6.3084e-02],
         [ 1.0610e+00, -6.2769e-02,  8.6025e-01, -2.7415e-01]]])
prob:
 tensor([[[0.0862, 0.9138, 0.0000, 0.00

## Intra-attention Mask

它是目标序列对原序列的一个 MSA，是 encoder 与 decoder 交互的部分。

由于 source seq 与 target seq 的长度存在不一致，若在某个位置上，任一方存在 pad 上去的元素，那么就是无效的 token，就是两者就不能计算相似度，因此这里需要一个 mask。

$QK^T$，其中 Q 来自于 target，shape 为 [B, tgt_seq_len, embed_dim], K 来自于 source，shape 为 [B. src_seq_len, embed_dim]，因此 score 的 shape 为 [B, tgt_seq_len, src_seq_len]，同样 mask 的 shape 也应当如此。

这里的实现与 encoder self-attention mask 很像。

In [14]:
valid_encoder_pos = torch.not_equal(src_seq, 0).to(torch.int32).unsqueeze(2)  # [B, src_seq_len, 1]
valid_decoder_pos = torch.not_equal(tgt_seq, 0).to(torch.int32).unsqueeze(2)  # [B, tgt_seq_len, 1]
logger.info(f'src_seq 中的有效位置:\n{valid_encoder_pos}')
logger.info(f'tgt_seq 中的有效位置:\n{valid_decoder_pos}')

2023-01-24 01:28:47.350 | INFO     | __main__:<module>:3 - src_seq 中的有效位置:
tensor([[[1],
         [1],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         [1]]], dtype=torch.int32)
2023-01-24 01:28:47.353 | INFO     | __main__:<module>:4 - tgt_seq 中的有效位置:
tensor([[[1],
         [1],
         [1],
         [1]],

        [[1],
         [1],
         [1],
         [0]]], dtype=torch.int32)


In [15]:
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1, 2))
logger.info(f'交叉的有效矩阵:\n{valid_cross_pos_matrix}')
# 在一个 tgt_seq_len × src_seq_len 的 matrix 中，[i, j] 处为 1 代表 tgt_seq 的 i 处 token 与 src_seq 的 j 处 token 是可以计算相似度的；为 0 则代表存在 pad 上去的 token，不能计算相似度

2023-01-24 01:28:49.476 | INFO     | __main__:<module>:2 - 交叉的有效矩阵:
tensor([[[1, 1, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 0, 0]],

        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [0, 0, 0, 0]]], dtype=torch.int32)


In [16]:
mask_cross_attn = valid_cross_pos_matrix == 0
logger.info(f'交叉的 mask:\n{mask_cross_attn}')
# 具体在使用上与之前的 encoder mask 是类似的，这里不再演示了

2023-01-24 01:28:52.753 | INFO     | __main__:<module>:2 - 交叉的 mask:
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [ True,  True,  True,  True]]])


## Decoder Self-Attention Mask

Transformer 的 Decoder 是自回归的，在预测一个位置的答案时，不能让 model 看到后面位置的答案，否则 model 就可以作弊了。

当 Decoder 预测第一个字符时，应该只给它看一个特殊字符；当 Decoder 预测第二个字符时，应该给它看“特殊字符和第一个字符”这两个字符...所以这里的 mask 应该是一个三角形的，在某一步的时候 mask 掉后面步的 target tokens。

In [17]:
tri_matrix = [torch.tril(torch.ones([L, L])) for L in tgt_len]  # 为每个 seq 生成一个三角阵
logger.info(f'三角阵:\n{tri_matrix}')

2023-01-24 01:28:55.061 | INFO     | __main__:<module>:2 - 三角阵:
[tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]]), tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])]


In [18]:
# 对 tri_matrix 做 padding
# 注意，这里不仅要对行做 pad，对列也要做 pad
valid_decoder_tri_matrix = torch.stack(
    [F.pad(m, [0, max(tgt_len) - m.size(0), 0, max(tgt_len) - m.size(0)]) for m in tri_matrix]
)
logger.info(f'decoder 有效矩阵:\n{valid_decoder_tri_matrix}')  # [B, T, T]
# 在一个 T×T 的 matrix 中，第 i 行为 1 的元素，代表在 step i 中可以计算相似度的 token，也就是具有相关性的 token

mask_decoder_self_attn = valid_decoder_tri_matrix == 0
logger.info(f'decoder mask matrix:\n{mask_decoder_self_attn}')
# 可以看到这里的掩码矩阵所做的 mask 就会让输入给 decoder 的数据变得有因果关系，从而可以自回归

2023-01-24 01:28:57.965 | INFO     | __main__:<module>:6 - decoder 有效矩阵:
tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [0., 0., 0., 0.]]])
2023-01-24 01:28:57.968 | INFO     | __main__:<module>:10 - decoder mask matrix:
tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])


In [19]:
# decoder mask 的使用示例
score = torch.randn([BATCH_SIZE, max(tgt_len), max(tgt_len)])
masked_score = score.masked_fill(mask_decoder_self_attn, -1e9)
prob = F.softmax(masked_score, -1)
print('tgt_len:\n', tgt_len)
print('masked_score:\n', masked_score)
print('prob:\n', prob)

tgt_len:
 tensor([4, 3], dtype=torch.int32)
masked_score:
 tensor([[[ 4.7183e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 4.4473e-01, -5.0157e-01, -1.0000e+09, -1.0000e+09],
         [ 1.5996e-01,  1.3268e+00, -1.9414e+00, -1.0000e+09],
         [-3.7091e-01,  7.9221e-01,  6.6882e-01, -8.6717e-01]],

        [[-7.1544e-02, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 2.7383e-03, -3.3346e-01, -1.0000e+09, -1.0000e+09],
         [-3.9580e-01, -3.6476e-01,  1.2710e+00, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])
prob:
 tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.7204, 0.2796, 0.0000, 0.0000],
         [0.2307, 0.7410, 0.0282, 0.0000],
         [0.1309, 0.4190, 0.3704, 0.0797]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5833, 0.4167, 0.0000, 0.0000],
         [0.1365, 0.1408, 0.7227, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])


## 构建 Self-Attention

公式：$Attention(Q, K, V) = softmax (\frac{QK^T}{\sqrt{d_k}})V$

In [20]:
def scaled_dot_product_attn(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attn_mask: Optional[torch.Tensor] = None
):
    """
    计算 multihead self-attention
    shape of Q,K,V: [B * num_head, seq_len, model_dim / num_head]
    注意，这里三者的 seq_len 不一定相同，比如在做 cross attention 时，有的来自于 target，有的来自于 source
    这里在 model_dim 再除以 num_head 是为了保持计算量与不使用 multi-head 时相同
    :return: _description_
    """
    assert len(query.size()) == 3
    assert len(key.size()) == 3
    assert len(value.size()) == 3
    assert query.size(0) == key.size(0)  # batch_size 相等
    assert query.size(2) == key.size(2)  # embed_dim 相等
    
    score = torch.bmm(query, key.transpose(-2, -1)) / torch.sqrt(MODEL_DIM)
    if attn_mask is not None:
        score = score.masked_fill(attn_mask, -1e9)
    prob = F.softmax(score, -1)
    context = torch.bmm(prob, value)  # 做 weighted sum 得到 context vector
    return context

## Masked loss


首先计算 loss：

In [21]:
logits = torch.randn(BATCH_SIZE, max(tgt_len), MAX_NUM_TGT_WORDS)  # model 的 predict 结果
label = torch.randint(low=0, high=MAX_NUM_TGT_WORDS, size=[BATCH_SIZE, max(tgt_len)])  # label

In [22]:
# 为了使用 cross_entropy，需要对 logits 做个简单的改动
logits = logits.transpose(1, 2)
loss = F.cross_entropy(logits, label, reduction='none')  # [B, seq_len]
logger.info(f'loss:\n{loss}')

2023-01-24 01:29:06.674 | INFO     | __main__:<module>:4 - loss:
tensor([[3.5744, 2.8208, 1.8123, 3.8194],
        [1.8657, 2.8765, 2.5268, 1.8193]])


由于不是每个 seq 的长度都是 `max(tgt_len)`，因此需要对 loss 也进行 mask：

In [23]:
mask_loss = (tgt_seq != 0).to(torch.int32)  # [B, seq_len]，元素为 1 的地方代表 loss 是有意义的
logger.info(f'mask of loss:\n{mask_loss}')

2023-01-24 01:29:10.114 | INFO     | __main__:<module>:2 - mask of loss:
tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]], dtype=torch.int32)


In [24]:
masked_loss = loss * mask_loss
logger.info(f'masked loss\n{masked_loss}')

2023-01-24 01:29:10.893 | INFO     | __main__:<module>:2 - masked loss
tensor([[3.5744, 2.8208, 1.8123, 3.8194],
        [1.8657, 2.8765, 2.5268, 0.0000]])


实际上在使用 PyTorch 时，`F.cross_entropy` 可以接收一个 `ignore_index` 的参数，你只需要把想要 mask 掉的 pad index 给它就好了。